# MATH 300: Numerical Analysis I Recitation

## Instructor: Liam Doherty

## Meeting Time/Place: W 11:00AM, Curtis 344

### Office Availability: MRC hours MTR 5p-7p (M 5-7 & T 5-6 FTF, others online), or in Korman 257 by appointment

I'm giving back Homework 1 today.

We'll start with some closing thoughts on root finding (Chapter 2 material). We have seen several methods for root finding and compared them (on the basis of their assumptions, order of convergence, etc.), and seen implementations of them from scratch using Julia. However, in practice, people typically use packages to solve these sorts of problems instead of coding their own algorithm (if the problem is generic enough, that is). We'll see how to use some of Julia's ecosystem to solve our own problems and simplify larger-scale codebases that we may produce with our own projects.

Suppose I want to find solutions to $e^{x} = x^{2}$ (i.e., solve the root finding problem $f(x) = 0$ where $f(x) = e^{x} - x^{2}$). We can solve this with the Bisection method using our code written previously:

In [ ]:
function bisection_manual(a::Real, b::Real, f::Function; abs_tol = 1e-5, max_iters = 100)
    # Check that the conditions of the algorithm are satisfied and we have not already located a zero
    @assert f(a)*f(b) ≤ 0 "Not in an interval with a sign change!"
    @assert (f(a) != 0) & (f(b) != 0) "One or both of the endpoints are zeros!"
    
    converged = false
    n = 0
    
    while !converged
        n += 1
        
        # Main step for algorithm
        p = (a + b)/2
        if f(a)*f(p) < 0
             b = p
        elseif f(b)*f(p) < 0
            a = p
        else
            println("Found exact zero at midpoint of iteration $(n)")
            return p
        end
        
        # Status updates
        println("a: $(a), b: $(b), n: $(n)")
        
        # Check convergence
        if b - a < abs_tol
            converged = true
            return p
        end
        
        # Return if algorithm does not converge
        if n == max_iters
            println("Did not converge in $(max_iters) iterations! Returning last value of p.")
            return p
        end
    end
end

In [ ]:
f(x) = exp(x) - x^2
bisection_manual(-1., 0., f)

This works just fine, but say we are working on a project that has root finding as a small component of a solution, and it is not the main focus of the problem. Then, instead of extending our codebase and writing an extra 30-40 lines of code to solve the root finding problem, we can just use somebody else's function that has already been written and drastically reduce our workload (and keep the focus of our project on the big result, instead of polluting our codebase with meaningless root finding code):

In [ ]:
using Roots
f(x) = exp(x) - x^2
p = Roots.bisection(f, -1., 0.) # Need the prefix of Roots. because function is not exported in source

Clearly, this is much easier to do from our end, because the code for bisection method is, in some sense, "hidden" behind the Roots package. We don't need to actually code the method ourselves, just importing Roots and calling the function from the package gives us the same result in 3 lines instead of ~30 lines. If you work on a project in the future outside of this class, check to see if somebody else wrote code you can use before you try to implement something yourself! It will save you time and potentially a headache as well!

A note here is that Roots.jl is not the _only_ way to compute roots with Julia. Another option would be using, say, Optim.jl (common nonlinear optimization toolbox) and minimizing the absolute value of $f$, i.e., solving the minimization problem $\newline \newline$
$$
p = \text{argmin}|f(x)|.
\newline
$$
The point here is simply that there are many ways to accomplish a specific goal, so you can choose whatever you are most comfortable with.

## Chapter 3 Content

Now we've wrapped up Chapter 2 and said some words about real world use of those concepts, so we'll move into Chapter 3. The chapter is about interpolation and polynomial approximation. The essence of "why we care" is that there are many scenarios in the real world where we have a discrete dataset of inputs and outputs and we would like to get a continuous (i.e., not discrete) function mapping that agrees completely on our dataset, and that we can then use to extract information not in the dataset (i.e., try to get the output of the mapping given an input we haven't seen before). This could be, for example, a dataset where we are given a city's population for a collection of years, and we would like to approximate the city's population at a year where there was no census taken (see problem 3.1.19).

The reason why we choose polynomials to approximate with is really a combination of two things: the Weierstrass Approximation Theorem, and the fact that calculus on polynomials is easy. The Weierstrass Approximation Theorem guarantees us that for well-behaved (i.e., continuous on a closed interval) functions, there exist sequences of polynomials that approximate the function to an arbitrary degree (within an arbitrary tolerance over the interval). Of course, the ease of calculus is self-explanatory; if we want to compute the integral of a "hard" function over some interval $[a, b]$, we can approximate it with a polynomial interpolant and take the integral of the interpolant (which can be done using linearity of integrals and power rule). This provides us with a way to integrate functions that don't have nice anti-derivatives (or any at all) as long as we can evaluate the function at a collection of nodes.

Now that some motivation has been given, let's look at a way to compute polynomials that agree with a function on a set of points, namely, the Lagrange interpolant. Given data $\newline \newline$
$$
\{(x_{0}, f(x_{0})), (x_{1}, f(x_{1})), \dots, (x_{n}, f(x_{n}))\},
\newline
$$
there is a unique (why?) polynomial of degree at most $n$ that passes through all of these points (i.e., agrees with the full dataset). Further, we can compute this polynomial with the formula $\newline \newline$
$$
P(x) = \sum_{k = 0}^{n}f(x_{k})L_{n,k}(x)
\newline
$$
where $\newline \newline$
$$
L_{n, k}(x) = \prod_{i = 0 \\ i \neq k}^{n}\frac{x - x_{i}}{x_{k} - x_{i}}.
\newline
$$
With a quick look at these formulas, you can see that $P(x_{j}) = f(x_{j})$, since $L_{n,k}(x_{j}) = 1$ if $x_{j} = x_{k}$, and $0$ otherwise. In a more compact notation, $\newline \newline$
$$
L_{n,k}(x_{j}) = \delta_{jk}.
\newline
$$
This is (more or less) a proof that the Lagrange interpolant actually agrees with our data (of course, if you wanted to prove it completely, more justification would be necessary). What this gives us is something to actualy compute with, so let's look at a problem to see it in action!

# Problem 3.1.1(a)

For the given function $f(x)$, let $x_{0} = 0$, $x_{1} = 0.6$, and $x_{2} = 0.9$. Construct interpolation polynomials of degree at most one and at most two to approximate $f(0.45)$ and find the absolute error. $\newline \newline$
$$
f(x) = \cos(x)
\newline
$$

## Solution:

We first need to evaluate $f$ at the nodes to give ourselves the full dataset to work with. For the first degree interpolant, we will only need $f(x_{0})$ and $f(x_{1})$, but for the second degree interpolant we will also require $f(x_{2})$. We have $\newline \newline$
$$
\cos(x_{0}) = \cos(0) = 1, \\
\cos(x_{1}) = \cos(0.6) \approx 0.8253, \\
\cos(x_{2}) = \cos(0.9) \approx 0.6216.
\newline
$$
So, our dataset is $\{(0, 1), (0.6, 0.8253), (0.9, 0.6216)\}$. For the first order interpolant, we need $\newline \newline$
$$
P(x) = \sum_{k = 0}^{1}f(x_{k})L_{1, k}(x),
\newline
$$
where $\newline \newline$
$$
L_{1, k}(x) = \prod_{i = 0 \\ i \neq k}^{1}\frac{x - x_{i}}{x_{k} - x_{i}}.
\newline
$$
Let's get the first term of our sum for $P$. We need $L_{1, 0}$ for that: $\newline \newline$
$$
L_{1, 0}(x) = \prod_{i = 0 \\ i \neq 0}^{1}\frac{x - x_{i}}{x_{0} - x_{i}} = \frac{x - x_{1}}{x_{0} - x_{1}} = \frac{x - 0.6}{0 - 0.6} = -\frac{5}{3}(x - \frac{3}{5}).
\newline
$$
So, our first term of the sum for $P$ is this result times $f(x_{0})$, which is $1$. Now, we'll compute the second term by first computing $L_{1, 1}$: $\newline \newline$
$$
L_{1, 1}(x) = \prod_{i = 0 \\ i \neq 1}^{1}\frac{x - x_{i}}{x_{1} - x_{i}} = \frac{x - x_{0}}{x_{1} - x_{0}} = \frac{x - 0}{0.6 - 0} = \frac{5}{3}x.
\newline
$$
And, of course, our second term of the sum is going to be $f(x_{1}) \approx 0.8253$ times this result. Our end result for the linear interpolant is that we have $\newline \newline$
$$
P(x) = -\frac{5}{3}(x - \frac{3}{5}) + 0.8253(\frac{5}{3}x) = -0.2912x + 1.
\newline
$$
Let's visualize this to see how the interpolant looks:

In [ ]:
using Plots; gr()
using Polynomials
using LaTeXStrings

f(x) = cos(x)
x_vals = [0., 0.6]
y_vals = f.(x_vals)
p = fit(x_vals, y_vals)

x_mesh = LinRange(-1, 1, 100)

plot(x_mesh, f.(x_mesh), label = L"$f(x)$", title = "Interpolating cos(x) with Linear Interpolant")
plot!(x_mesh, p.(x_mesh), label = L"$P(x)$")
scatter!([(0, 1), (0.6, 0.8253)], label = "Interpolating Nodes")

In [ ]:
println("Approximation at 0.45: $(p(0.45))")
println("Absolute error: $(abs(p(0.45) - cos(0.45)))")

(Notice the use of Polynomials.jl and the fit function above, it saves some coding, in the spirit of the first part of today's notebook!)

Now, we want to compute the second degree interpolant (i.e., the quadratic interpolant). Unfortunately, we have to recompute the summands, since now we have the $L_{n, k}$ being of order 2 instead of order 1 (i.e., our Lagrange basis polynomials are now quadratic, rather than linear). We get $\newline \newline$
$$
P(x) = \sum_{k = 0}^{2}f(x_{k})L_{2, k}
\newline
$$
For the first summand, we have $\newline \newline$
$$
\begin{align*}
    f(x_{0})L_{2, 0}(x) = L_{2, 0}(x) &= \prod_{i = 0 \\ i \neq 0}^{2}\frac{x - x_{i}}{x_{0} - x_{i}} \\
    &= \frac{x - x_{1}}{x_{0} - x_{1}} \cdot \frac{x - x_{2}}{x_{0} - x_{2}} \\
    &= \frac{x - 0.6}{0 - 0.6} \cdot \frac{x - 0.9}{0 - 0.9} \\
    &= \frac{1}{0.54}(x^{2} - 1.5x + 0.54) \\
    &\approx 1.85x^{2} - 2.77x + 1.
\end{align*}
\newline
$$
For the second summand, we get $\newline \newline$
$$
\begin{align*}
    f(x_{1})L_{2, 1}(x) &= 0.8253L_{2, 1}(x) \\
    &= 0.8253\prod_{i = 0 \\ i \neq 1}^{2}\frac{x - x_{i}}{x_{1} - x_{i}} \\
    &= 0.8253 \cdot \frac{x - x_{0}}{x_{1} - x_{0}} \cdot \frac{x - x_{2}}{x_{1} - x_{2}} \\
    &= 0.8253\frac{x - 0}{0.6 - 0} \cdot \frac{x - 0.9}{0.6 - 0.9} \\
    &= -\frac{0.8253}{0.18}(x^{2} - 0.9x) \\
    &\approx -4.585x^{2} + 4.114x.
\end{align*}
\newline
$$
For the final summand, we obtain $\newline \newline$
$$
\begin{align*}
    f(x_{2})L_{2, 2}(x) &= 0.6216L_{2, 2}(x) \\
    &= 0.6216\prod_{i = 0 \\ i \neq 2}^{2}\frac{x - x_{i}}{x_{2} - x_{i}} \\
    &= 0.6216 \cdot \frac{x - x_{0}}{x_{2} - x_{0}} \cdot \frac{x - x_{1}}{x_{2} - x_{1}} \\
    &= 0.6216\frac{x - 0}{0.9 - 0} \cdot \frac{x - 0.6}{0.9 - 0.6} \\
    &= \frac{0.6216}{0.27}(x^{2} - 0.6x) \\
    &\approx 2.302x^{2} - 1.38x.
\end{align*}
\newline
$$
At last, we are in the right position to obtain $P(x)$. We add these results together to find $\newline \newline$
$$
\begin{align*}
    P(x) &= \sum_{k = 0}^{2}f(x_{k})L_{2, k}(x) \\
    &= (1.85x^{2} - 2.77x + 1) + (-4.585x^{2} + 4.114x) + (2.302x^{2} - 1.38x) \\
    &= -0.433x^{2} - 0.036x + 1. 
\end{align*}
\newline
$$
We can plot this again to see how it looks compared to the graph of $\cos(x)$:

In [ ]:
f(x) = cos(x)
x_vals = [0., 0.6, 0.9]
y_vals = f.(x_vals)
q = fit(x_vals, y_vals)

x_mesh = LinRange(-2, 2, 100)

plot(x_mesh, f.(x_mesh), label = L"$f(x)$", title = "Interpolating cos(x) with Quadratic Interpolant", legend =:bottomleft)
plot!(x_mesh, q.(x_mesh), label = L"$Q(x)$")
scatter!([(0, 1), (0.6, 0.8253), (0.9, 0.6216)], label = "Interpolating Nodes")

In [ ]:
println("Approximation at 0.45: $(q(0.45))")
println("Absolute error: $(abs(q(0.45) - cos(0.45)))")

# Problem 3.1.3(a)

Use Theorem 3.3 to provide bounds for the error in the interpolants above.

## Solution:

Recall that Theorem 3 says that given a set of $n + 1$ distinct nodes $\{x_{k}\}_{k = 0}^{n}$ and a function that is $n + 1$ times continuously differentiable all in an interval $[a, b]$, then there exists a $\xi$ between the leftmost and rightmost nodes so that $\newline \newline$
$$
f(x) = P(x) + \frac{f^{(n + 1)}(\xi)}{(n + 1)!}\prod_{k = 0}^{n}(x - x_{k}).
\newline
$$
(Notice the very similar looking form to the Taylor error!) From this, we get an error bound of (assuming the nodes are appropriately ordered, i.e., $x_{0} < x_{1} < \dots < x_{n}$) $\newline \newline$
$$
|f(x) - P(x)| \leq \max_{x \in [x_{0}, x_{n}]}\Big|\frac{f^{(n + 1)}(x)}{(n + 1)!}\prod_{k = 0}^{n}(x - x_{k})\Big|
\newline
$$
We will only do the analysis for the degree one interpolant; the second degree interpolant is similar (with a bit more arithmetic along the way). For the degree one interpolant, $n = 1$, so we need the second derivative of $f$, which is $-\cos(x)$. Maximizing the absolute value of this over $[0, 0.6]$, we get a bound of $1$. This tells us that our coefficient before the product has a bound of $\newline \newline$
$$
\Big|\frac{f^{(n + 1)}(x)}{(n + 1)!}\Big| \leq \frac{1}{2}.
\newline
$$
For the polynomial piece, we have the maximization problem $\newline \newline$
$$
\max_{[0, 0.6]}\Big|(x - 0)(x - 0.6)\Big| = \max_{[0, 0.6]}\Big|x^{2} - 0.6x\Big|.
\newline
$$
This can be done with the usual calculus techniques. The derivative of the polynomial is $q'(x) = 2x - 0.6$, so our critical point is $p = 0.3$. It is easy to check (i.e., first or second derivative test) that $p$ is a mimimum, and $q(p) = (0.3)^{2} - 0.6(0.3) = -0.09$. Of course, we must also check the value of the function at the endpoints, but by the definition of the function $f$, both of these values are zero. Hence, our bound on the product is $\newline \newline$
$$
\max_{[0, 0.9]}\Big|(x - 0)(x - 0.6)\Big| \leq |q(0.3)| = 0.09.
\newline
$$
This tells us that our overall bound on the error on $[0, 0.6]$ is $\newline \newline$
$$
|f(x) - P(x)| \leq \max_{[0, 0.6]}\Big|\frac{f''(x)}{2!}(x - 0)(x - 0.6)\Big| \leq \frac{1}{2} \cdot 0.09 = 0.045.
\newline
$$
We can check this bound with Julia's norm function (which requires LinearAlgebra):

In [ ]:
using LinearAlgebra

f(x) = cos(x)
x_vals = [0., 0.6]
y_vals = f.(x_vals)
r = fit(x_vals, y_vals)

x_mesh = LinRange(0, 0.6, 100)
error = abs.(f.(x_mesh) - r.(x_mesh))

println("Maximum error over the interval [0, 0.9]: $(norm(error, Inf))")